In [1]:
import pandas as pd

In [2]:
nli_df = pd.read_csv('/Users/noamkashi/Documents/תוצאות ספרית שאטרסטום/merged_books_data.csv')
google_df = pd.read_csv('/Users/noamkashi/Documents/תוצאות ספרית שאטרסטום/merged_results_google.csv')
print(google_df.columns)
print(nli_df.columns)

Index(['original_index', 'book name', 'result_rank', 'title', 'authors',
       'published_date', 'page_count', 'categories', 'language',
       'preview_link', 'google_books_id', 'description', 'index', 'level_0'],
      dtype='object')
Index(['book name', 'nli_search', 'book_index', 'nli_id', 'nli_language',
       'nli_identifier', 'nli_creator', 'nli_date', 'nli_accessRights',
       'nli_title', 'nli_type', 'nli_recordid', 'nli_source', 'nli_linkToMarc',
       'nli_thumbnail', 'nli_download', 'nli_relation',
       'nli_non_standard_date', 'nli_publisher', 'nli_contributor',
       'nli_format', 'nli_subject', 'nli_wgs84_pos', 'nli_isbn', 'source',
       'index'],
      dtype='object')


In [37]:
# First, add a source column to each dataframe
google_df['row_source'] = 'google'
nli_df['row_source'] = 'nli'

# Use pandas concat to stack the dataframes vertically (like UNION)
merged_df = pd.concat([google_df, nli_df], ignore_index=True)

# If you want to sort by book name to have similar books near each other
merged_df = merged_df.sort_values(by='book name')

# Optionally add a rank column
merged_df['rank'] = 0.0

In [30]:
print(merged_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1706027 entries, 0 to 1706026
Data columns (total 40 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   original_index         float64
 1   book name              object 
 2   result_rank            float64
 3   title                  object 
 4   authors                object 
 5   published_date         float64
 6   page_count             float64
 7   categories             object 
 8   language               object 
 9   preview_link           object 
 10  google_books_id        object 
 11  description            object 
 12  index_google           float64
 13  level_0                float64
 14  nli_search             object 
 15  book_index             float64
 16  nli_id                 object 
 17  nli_language           object 
 18  nli_identifier         object 
 19  nli_creator            object 
 20  nli_date               object 
 21  nli_accessRights       object 
 22  nli_title         

In [38]:
def count_matching_words_improved(book_name, target_text):
    # Handle NaN values
    if pd.isna(book_name) or pd.isna(target_text):
        return 0
    
    # Convert to lowercase and split into words
    book_words = set(book_name.lower().split())
    target_words = target_text.lower().split()
    
    matches = 0
    
    # For each word in the book name
    for book_word in book_words:
        # Check for direct match
        if book_word in target_words:
            matches += 1
        # Check for match without initial ה if the word starts with ה and is long enough
        elif book_word.startswith('ה') and len(book_word) > 2:
            word_without_h = book_word[1:]  # Remove the first letter (ה)
            if word_without_h in target_words:
                matches += 1
        # Check if target words have ה prefix that book_word doesn't have
        else:
            prefixed_word = 'ה' + book_word
            if prefixed_word in target_words:
                matches += 1
    
    return matches

# Define the word matching function with a different name
def count_matching_words_authors(book_name, target_text):
    # Handle NaN values
    if pd.isna(book_name) or pd.isna(target_text):
        return 0
    
    # Convert to lowercase and split into words
    book_words = set(book_name.lower().split())
    target_words = target_text.lower().split()
    
    # Count matches
    matches = sum(1 for word in book_words if word in target_words)
    return matches

# Create a function to apply to each row for authors and nli_creator with a unique name
def update_rank_for_author_matches(row):
    additional_rank = 0
    
    # Check authors column (from google_df)
    if not pd.isna(row.get('authors')):
        matches = count_matching_words_authors(row['book name'], row['authors'])
        additional_rank += matches * 0.5
    
    # Check nli_creator column (from nli_df)
    if not pd.isna(row.get('nli_creator')):
        matches = count_matching_words_authors(row['book name'], row['nli_creator'])
        additional_rank += matches * 0.5
    
    # Return the additional rank to be added
    return additional_rank
    
# Define the list of words to check for
publisher_words = ["ירושלים", "חיפה", "תל אביב"]

# Function to check if any of the specified words are in the publisher text
def check_publisher_words(publisher_text):
    if pd.isna(publisher_text):
        return False
    
    # Convert to string in case it's not already
    publisher_text = str(publisher_text)
    
    # Check if any of the words are in the text
    for word in publisher_words:
        if word in publisher_text:
            return True
    
    return False
# Define the list of words to check for
title_words = ["מתוך", "חלק"]

# Function to check if any of the specified words are in the title text
def check_title_words(row):
    # Initialize result
    contains_word = False
    
    # Check title column
    if not pd.isna(row.get('title')):
        title_text = str(row['title'])
        for word in title_words:
            if word in title_text:
                contains_word = True
                break
    
    # Check nli_title column if we haven't found a match yet
    if not contains_word and not pd.isna(row.get('nli_title')):
        nli_title_text = str(row['nli_title'])
        for word in title_words:
            if word in nli_title_text:
                contains_word = True
                break
    
    return contains_word
format_words = ["כרך", "כרכים"]

def check_format_words(format_text):
    if pd.isna(format_text):
        return False
    
    # Convert to string in case it's not already
    format_text = str(format_text)
    
    # Check if any of the words are in the text
    for word in format_words:
        if word in format_text:
            return True
    
    return False
#ran all ranking

# Apply the function to calculate additional rank for each row
additional_ranks_authors = merged_df.apply(update_rank_for_author_matches, axis=1)

# Add the additional ranks to the existing rank column
merged_df['rank'] += additional_ranks_authors

# Apply the function to calculate additional rank for each row
additional_ranks = merged_df.apply(update_rank_for_matching_words, axis=1)

# Add the additional ranks to the existing rank column
merged_df['rank'] += additional_ranks

#add 1 pint for nli sourceֿ
merged_df.loc[merged_df['row_source'] == 'nli', 'rank'] += 1.0

merged_df.loc[merged_df['nli_publisher'].apply(check_publisher_words), 'rank'] -= 1
merged_df.loc[merged_df.apply(check_title_words, axis=1), 'rank'] -= 1
merged_df.loc[merged_df['nli_format'].apply(check_format_words), 'rank'] += 1




In [39]:
# 1. Remove rows where 'book name' contains 'הדור'
filtered_df = merged_df[~merged_df['book name'].str.contains('הדור', na=False)]

# 2. Sort by rank in descending order (highest rank first)
filtered_df = filtered_df.sort_values(by='rank', ascending=False)

# 3. Then remove duplicates based on the specified columns
# This keeps the first occurrence of each duplicate set (which will be the one with highest rank due to sorting)
deduplicated_df = filtered_df.drop_duplicates(
    subset=['title', 'authors', 'published_date', 'page_count', 'categories'],
    keep='first'
)

In [23]:
# Take the first 600 rows
top_600_df = deduplicated_df.head(600)

# Save to CSV
top_600_df.to_csv('top_600_merged_books.csv', index=False)

In [45]:
# Create an empty list to store DataFrames
top_dfs = []

# Process Google data
google_top = merged_df[merged_df['row_source'] == 'google'].groupby('book name').apply(
    lambda x: x.nlargest(2, 'rank')
).reset_index(drop=True)
top_dfs.append(google_top)

# Process NLI data
nli_top = merged_df[merged_df['row_source'] == 'nli'].groupby('book name').apply(
    lambda x: x.nlargest(2, 'rank')
).reset_index(drop=True)
top_dfs.append(nli_top)

# Combine the top entries from both sources
final_df = pd.concat(top_dfs, ignore_index=True)

# Sort by book name and then by rank in descending order
final_df = final_df.sort_values(by=['book name', 'rank'], ascending=[True, False])

# Reset index
final_df = final_df.reset_index(drop=True)

# Save to CSV
final_df.to_csv('final_top2_per_source_per_book.csv', index=False)

/var/folders/p1/l8nqtyf91pb7vks5m2l_jpgm0000gn/T/ipykernel_13912/3823076772.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  google_top = merged_df[merged_df['row_source'] == 'google'].groupby('book name').apply(
/var/folders/p1/l8nqtyf91pb7vks5m2l_jpgm0000gn/T/ipykernel_13912/3823076772.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nli_top = merged_df[merged_df['row_source'] == 'nli'].groupby('bo

In [48]:
# Select only the desired columns from the final dataframe
selected_columns = [
    'book name',
    'title',
    'nli_title',
    'authors',
    'rank',
    'published_date',
    'nli_creator',
    'row_source',
]

# Create the new dataframe with only these columns
simplified_df = final_df[selected_columns]

# Save to CSV
simplified_df.to_csv('simplified_book_data.csv', index=False)

In [5]:
import requests
import urllib.parse
import json
import time

API_KEYS = [
    'ohSupNWHPC2ncMlosQYmMYzHp7GwmZ4HKY0PXoMX',
    'mZcuSKOCiaboR9ctxclfeMACDtXAgoYQL3cdTRWS',
    't2A91mZgPnhShnDsvGWZWmnj9s7OsTDW072xrg3e',
    'm5TELRWYrhqJmVj10VEjUQVPTIduz02QiFpFtWEh',
    'Od48KLpdU4sQK4k4d5Lf0rtbcTP1NGpVmJkXurM7'
]
BASE_URL = 'https://api.nli.org.il/openlibrary/search'

current_key_index = 0  # Start with the first key

def get_current_api_key():
    return API_KEYS[current_key_index]

def switch_to_next_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(API_KEYS)
    print(f"Switching to API key: {get_current_api_key()[:5]}...")
    return get_current_api_key()

def search_nli(search_string, retry_count=0):
    """
    Search the NLI API for a specific string and return the response
    """
    if retry_count >= len(API_KEYS):
        print(f"All API keys are rate limited. Please try again later.")
        return None

    current_api_key = get_current_api_key()

    try:
        # Encode the search string for URL
        encoded_search = urllib.parse.quote(search_string)

        # Construct the URL - using 'any,contains' to search across all fields
        url = f"{BASE_URL}?api_key={current_api_key}&query=any,contains,{encoded_search}&bulkSize=30&index=0&sort=rank"

        print(f"Request URL: {url}")

        # Make the request
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429 or "OVER_RATE_LIMIT" in response.text:
            print(f"Rate limit exceeded for key {current_api_key[:5]}...")
            switch_to_next_api_key()
            # Retry with the new key
            return search_nli(search_string, retry_count + 1)
        else:
            # Print error information
            error_msg = f"Error: {response.status_code} - {response.text}"
            print(f"Search term: '{search_string}' - {error_msg}")
            return None
    except Exception as e:
        error_msg = f"Exception occurred: {e}"
        print(f"Search term: '{search_string}' - {error_msg}")
        return None

# The specific ID you want to search
search_id = "990020209770205171"

print(f"Searching for ID: {search_id}")
print(f"Starting with API key: {get_current_api_key()[:5]}...")

# Perform the search
result = search_nli(search_id)

# Process and display the results
if result:
    # Check the type of result and process accordingly
    if isinstance(result, dict):
        total_found = result.get('total', 0)
        print(f"\nSearch complete. Found {total_found} results.")

        # Get the docs part if it exists
        if 'docs' in result and result['docs']:
            docs = result['docs']
            print(f"Number of documents returned: {len(docs)}")

            # Process the first document
            if docs:
                print("\nDetails of first result:")
                first_doc = docs[0]

                # Display key information if available
                print(f"Title: {first_doc.get('title', 'N/A')}")
                print(f"Author: {', '.join(first_doc.get('author', ['N/A'])) if 'author' in first_doc and isinstance(first_doc['author'], list) else 'N/A'}")
                print(f"Year: {first_doc.get('year', 'N/A')}")
                print(f"Record ID: {first_doc.get('recordid', 'N/A')}")

                # Save the full result to a JSON file
                with open(f'nli_search_result_{search_id}.json', 'w', encoding='utf-8') as f:
                    json.dump(docs, f, ensure_ascii=False, indent=2)
                print(f"\nFull results saved to nli_search_result_{search_id}.json")
        else:
            print("No documents found in the response.")
    elif isinstance(result, list):
        # The result is already a list of documents
        docs = result
        print(f"\nSearch complete. Found {len(docs)} results.")

        if docs:
            print("\nDetails of first result:")
            first_doc = docs[0]

            # Display key information if available (using dict.get() for safety)
            print(f"Title: {first_doc.get('title', 'N/A')}")
            print(f"Author: {', '.join(first_doc.get('author', ['N/A'])) if 'author' in first_doc and isinstance(first_doc['author'], list) else 'N/A'}")
            print(f"Year: {first_doc.get('year', 'N/A')}")
            print(f"Record ID: {first_doc.get('recordid', 'N/A')}")

            # Save the full result to a JSON file
            with open(f'nli_search_result_{search_id}.json', 'w', encoding='utf-8') as f:
                json.dump(docs, f, ensure_ascii=False, indent=2)
            print(f"\nFull results saved to nli_search_result_{search_id}.json")
        else:
            print("No documents found in the response.")
    else:
        print("No documents found in the response.")
else:
    print("The search did not return any results or encountered an error.")

Searching for ID: 990020209770205171
Starting with API key: ohSup...
Request URL: https://api.nli.org.il/openlibrary/search?api_key=ohSupNWHPC2ncMlosQYmMYzHp7GwmZ4HKY0PXoMX&query=any,contains,990020209770205171&bulkSize=30&index=0&sort=rank

Search complete. Found 1 results.

Details of first result:
Title: N/A
Author: N/A
Year: N/A
Record ID: N/A

Full results saved to nli_search_result_990020209770205171.json


In [16]:
import requests
import urllib.parse
import json
import time
import pandas as pd
import re
import csv

# Your API keys
API_KEYS = [
    'mZcuSKOCiaboR9ctxclfeMACDtXAgoYQL3cdTRWS',
    't2A91mZgPnhShnDsvGWZWmnj9s7OsTDW072xrg3e',
    'm5TELRWYrhqJmVj10VEjUQVPTIduz02QiFpFtWEh',
    'Od48KLpdU4sQK4k4d5Lf0rtbcTP1NGpVmJkXurM7'
]
BASE_URL = 'https://api.nli.org.il/openlibrary/search'

current_key_index = 0  # Start with the first key

def get_current_api_key():
    return API_KEYS[current_key_index]

def switch_to_next_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(API_KEYS)
    print(f"Switching to API key: {get_current_api_key()[:5]}...")
    return get_current_api_key()

def search_nli(search_string, retry_count=0):
    """
    Search the NLI API for a specific string and return the response
    """
    if retry_count >= len(API_KEYS):
        print(f"All API keys are rate limited. Please try again later.")
        return None

    current_api_key = get_current_api_key()

    try:
        # Encode the search string for URL
        encoded_search = urllib.parse.quote(search_string)

        # Construct the URL - using 'any,contains' to search across all fields
        url = f"{BASE_URL}?api_key={current_api_key}&query=any,contains,{encoded_search}&bulkSize=30&index=0&sort=rank"

        print(f"Searching for ID: {search_string}")

        # Make the request
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429 or "OVER_RATE_LIMIT" in response.text:
            print(f"Rate limit exceeded for key {current_api_key[:5]}...")
            switch_to_next_api_key()
            # Retry with the new key
            return search_nli(search_string, retry_count + 1)
        else:
            # Print error information
            error_msg = f"Error: {response.status_code} - {response.text}"
            print(f"Search term: '{search_string}' - {error_msg}")
            return None
    except Exception as e:
        error_msg = f"Exception occurred: {e}"
        print(f"Search term: '{search_string}' - {error_msg}")
        return None

def clean_value(value):
    # Remove any special formatting codes like $$Q, $$V, etc.
    return re.sub(r'\$\$[A-Z][^$]*', ' ', value).strip()

def parse_contributors(contrib_str):
    # Split the contributors by the pattern "$$Q"
    contributors = []
    parts = contrib_str.split('$$Q')
    for part in parts:
        if part.strip():
            contributors.append(clean_value(part))
    return contributors

def convert_nli_json_to_dict(data):
    # Create a dictionary to store the extracted data
    result = {}

    if data and len(data) > 0:
        item = data[0]

        # Extract basic metadata
        result['id'] = item.get('@id', '')

        # Title
        title_list = item.get('http://purl.org/dc/elements/1.1/title', [])
        if title_list:
            result['title'] = title_list[0].get('@value', '')

        # Type
        type_list = item.get('http://purl.org/dc/elements/1.1/type', [])
        if type_list:
            result['type'] = type_list[0].get('@value', '')

        # Record ID
        record_id_list = item.get('http://purl.org/dc/elements/1.1/recordid', [])
        if record_id_list:
            result['record_id'] = record_id_list[0].get('@value', '')

        # Language
        lang_list = item.get('http://purl.org/dc/elements/1.1/language', [])
        if lang_list:
            result['language'] = lang_list[0].get('@value', '')

        # Subject
        subject_list = item.get('http://purl.org/dc/elements/1.1/subject', [])
        if subject_list:
            result['subject'] = subject_list[0].get('@value', '')

        # Publisher
        publisher_list = item.get('http://purl.org/dc/elements/1.1/publisher', [])
        if publisher_list:
            result['publisher'] = publisher_list[0].get('@value', '')

        # Date
        date_list = item.get('http://purl.org/dc/elements/1.1/non_standard_date', [])
        if date_list:
            result['date'] = date_list[0].get('@value', '')

        # Contributors - parse the complex string into a list and join with semicolons for CSV
        contrib_list = item.get('http://purl.org/dc/elements/1.1/contributor', [])
        if contrib_list:
            contrib_str = contrib_list[0].get('@value', '')
            result['contributors'] = '; '.join(parse_contributors(contrib_str))

        # Format
        format_list = item.get('http://purl.org/dc/elements/1.1/format', [])
        if format_list:
            result['format'] = format_list[0].get('@value', '')

        # Thumbnail
        thumbnail_list = item.get('http://purl.org/dc/elements/1.1/thumbnail', [])
        if thumbnail_list:
            result['thumbnail'] = thumbnail_list[0].get('@value', '')

        # Access Rights
        access_list = item.get('http://purl.org/dc/elements/1.1/accessRights', [])
        if access_list:
            result['access_rights'] = access_list[0].get('@value', '')

        # Link to MARC
        marc_list = item.get('http://purl.org/dc/elements/1.1/linkToMarc', [])
        if marc_list:
            result['link_to_marc'] = marc_list[0].get('@id', '')

    return result

# Main function to process all IDs
def process_missing_data(csv_file_path):
    # Read the CSV file with missing data
    df = pd.read_csv(csv_file_path)

    # Check if 'id' column exists
    if 'id' not in df.columns:
        print("Error: 'id' column not found in the CSV file.")
        return

    # Initialize a list to store the results
    results = []

    # Process each ID in the CSV file
    for index, row in df.iterrows():
        search_id = str(row['id']).strip()
        print(f"\nProcessing ID {index+1}/{len(df)}: {search_id}")

        # Skip empty IDs
        if not search_id or pd.isna(search_id):
            print("Skipping empty ID")
            results.append({})
            continue

        # Search the NLI API
        result = search_nli(search_id)

        # If we got a result, convert it to a dict
        if result:
            # Check if the result is a dict with 'docs' or a list
            if isinstance(result, dict) and 'docs' in result and result['docs']:
                converted_data = convert_nli_json_to_dict(result['docs'])
                results.append(converted_data)
            elif isinstance(result, list) and result:
                converted_data = convert_nli_json_to_dict(result)
                results.append(converted_data)
            else:
                print(f"No valid data found for ID: {search_id}")
                results.append({})
        else:
            print(f"No result found for ID: {search_id}")
            results.append({})

        # Add a small delay to avoid overwhelming the API
        time.sleep(1)

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Merge the original dataframe with the results
    if not results_df.empty:
        # Remove columns from original df that will be replaced by results_df
        # to avoid duplicate columns with suffix '_x' and '_y'
        for col in results_df.columns:
            if col in df.columns and col != 'id':
                df = df.drop(columns=[col])

        # Merge the dataframes
        merged_df = pd.concat([df, results_df], axis=1)
    else:
        merged_df = df

    # Save the result to a new CSV file
    output_file = csv_file_path.replace('.csv', '_enriched.csv')
    merged_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\nEnriched data saved to: {output_file}")

    return merged_df

# Run the main function
if __name__ == "__main__":
    # File path
    csv_file_path = '/Users/noamkashi/Downloads/last_books_for_real - Sheet1.csv'

    print(f"Starting with API key: {get_current_api_key()[:5]}...")

    # Process the data
    enriched_data = process_missing_data(csv_file_path)

    print("Process completed successfully!")

Starting with API key: mZcuS...

Processing ID 1/21: 990020950010205171
Searching for ID: 990020950010205171

Processing ID 2/21: 990020636480205171
Searching for ID: 990020636480205171

Processing ID 3/21: 990020636480205171
Searching for ID: 990020636480205171

Processing ID 4/21: 990020636480205171
Searching for ID: 990020636480205171

Processing ID 5/21: 990020636480205171
Searching for ID: 990020636480205171

Processing ID 6/21: 990020636480205171
Searching for ID: 990020636480205171

Processing ID 7/21: 990050547310205171
Searching for ID: 990050547310205171

Processing ID 8/21: 990018647290205171
Searching for ID: 990018647290205171

Processing ID 9/21: 990018647290205171
Searching for ID: 990018647290205171

Processing ID 10/21: 990018647290205171
Searching for ID: 990018647290205171

Processing ID 11/21: 990052861010205171
Searching for ID: 990052861010205171

Processing ID 12/21: 990019740380205171
Searching for ID: 990019740380205171

Processing ID 13/21: 990018084370205171


In [9]:
import pandas as pd

def match_and_merge_csvs(first_csv_path, second_csv_path):
    # Read both CSV files
    print(f"Reading files...")
    df1 = pd.read_csv(first_csv_path)
    df2 = pd.read_csv(second_csv_path)

    print(f"First CSV shape: {df1.shape}")
    print(f"Second CSV shape: {df2.shape}")

    # Columns to match (all columns except 'book name' and 'nli_recordid')
    match_columns = ['title', 'nli_title', 'authors', 'rank', 'published_date', 'nli_creator', 'row_source']

    # Check if all required columns exist in both dataframes
    missing_cols_df1 = [col for col in match_columns if col not in df1.columns]
    missing_cols_df2 = [col for col in match_columns if col not in df2.columns]

    if missing_cols_df1:
        print(f"Error: Columns {missing_cols_df1} missing in first CSV.")
        return

    if missing_cols_df2:
        print(f"Error: Columns {missing_cols_df2} missing in second CSV.")
        return

    if 'nli_recordid' not in df2.columns:
        print(f"Error: Column 'nli_recordid' missing in second CSV.")
        return

    # Create a new 'id' column in df1 initialized with None/NaN
    df1['id'] = None

    # Count matches
    match_count = 0

    print("Finding matches and merging data...")

    # For each row in df1, find matching rows in df2
    for index1, row1 in df1.iterrows():
        # Create a mask for matching all specified columns
        mask = pd.Series(True, index=df2.index)

        for col in match_columns:
            # Handle NaN values specially
            if pd.isna(row1[col]):
                mask = mask & df2[col].isna()
            else:
                mask = mask & (df2[col] == row1[col])

        # Find matching rows in df2
        matching_rows = df2[mask]

        # If there's a match, get the nli_recordid from df2
        if not matching_rows.empty:
            match_count += 1
            # Take the first match if multiple exist
            df1.at[index1, 'id'] = matching_rows.iloc[0]['nli_recordid']

    print(f"Found {match_count} matches out of {len(df1)} rows.")

    # Save the updated df1 as a new CSV
    output_path = first_csv_path.replace('.csv', '_with_ids.csv')
    df1.to_csv(output_path, index=False, encoding='utf-8')
    print(f"Updated data saved to: {output_path}")

    return df1

# File paths
first_csv_path = '/Users/noamkashi/Downloads/missing data - nli books.csv'
second_csv_path = '/Users/noamkashi/Documents/Strashum Library/final_top2_per_source_per_book.csv'

# Run the function
result_df = match_and_merge_csvs(first_csv_path, second_csv_path)

Reading files...
First CSV shape: (537, 8)
Second CSV shape: (4406, 40)
Finding matches and merging data...
Found 534 matches out of 537 rows.
Updated data saved to: /Users/noamkashi/Downloads/missing data - nli books_with_ids.csv


In [22]:
import requests
import urllib.parse
import json
import csv
import re
import time
import os

# Your API keys
API_KEYS = [
    'mZcuSKOCiaboR9ctxclfeMACDtXAgoYQL3cdTRWS',
    't2A91mZgPnhShnDsvGWZWmnj9s7OsTDW072xrg3e',
    'm5TELRWYrhqJmVj10VEjUQVPTIduz02QiFpFtWEh',
    'Od48KLpdU4sQK4k4d5Lf0rtbcTP1NGpVmJkXurM7'
]
BASE_URL = 'https://api.nli.org.il/openlibrary/search'

current_key_index = 0  # Start with the first key

def get_current_api_key():
    return API_KEYS[current_key_index]

def switch_to_next_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(API_KEYS)
    print(f"Switching to API key: {get_current_api_key()[:5]}...")
    return get_current_api_key()

def search_nli(search_string, bulk_size=30, retry_count=0):
    """
    Search the NLI API for a specific string and return all results
    """
    if retry_count >= len(API_KEYS):
        print(f"All API keys are rate limited. Please try again later.")
        return None

    current_api_key = get_current_api_key()

    try:
        # Encode the search string for URL
        encoded_search = urllib.parse.quote(search_string)

        # Construct the URL - using 'any,contains' to search across all fields
        url = f"{BASE_URL}?api_key={current_api_key}&query=any,contains,{encoded_search}&bulkSize={bulk_size}&index=0&sort=rank"

        print(f"Searching for: {search_string}")
        print(f"Request URL: {url}")

        # Make the request
        response = requests.get(url)

        # Print the raw response for debugging
        print(f"Response status code: {response.status_code}")
        print(f"Response content type: {response.headers.get('Content-Type', 'unknown')}")

        # Check if the request was successful
        if response.status_code == 200:
            try:
                result = response.json()
                # Print response structure for debugging
                print(f"Response contains keys: {list(result.keys()) if isinstance(result, dict) else 'Not a dictionary'}")

                # Check the structure - NLI sometimes puts docs in different places
                if isinstance(result, dict):
                    if 'docs' in result:
                        print(f"Found {len(result['docs'])} documents in 'docs' key")
                    elif 'records' in result:
                        print(f"Found {len(result['records'])} documents in 'records' key")
                        # Adjust the structure for our processing
                        result['docs'] = result['records']
                    elif 'results' in result:
                        print(f"Found {len(result['results'])} documents in 'results' key")
                        # Adjust the structure for our processing
                        result['docs'] = result['results']
                    else:
                        print("Response structure is not as expected. Raw data (first 500 chars):")
                        print(json.dumps(result)[:500] + "...")
                else:
                    print("Response is not a dictionary. Raw data (first 500 chars):")
                    print(json.dumps(result)[:500] + "...")

                return result

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON response: {e}")
                print("Raw response (first 500 chars):")
                print(response.text[:500] + "...")
                return None

        elif response.status_code == 429 or "OVER_RATE_LIMIT" in response.text:
            print(f"Rate limit exceeded for key {current_api_key[:5]}...")
            switch_to_next_api_key()
            # Retry with the new key
            return search_nli(search_string, bulk_size, retry_count + 1)
        else:
            # Print error information
            error_msg = f"Error: {response.status_code} - {response.text[:200]}"
            print(f"Search term: '{search_string}' - {error_msg}")
            return None
    except Exception as e:
        error_msg = f"Exception occurred: {e}"
        print(f"Search term: '{search_string}' - {error_msg}")
        return None

def clean_value(value):
    if not value:
        return ""
    # Remove any special formatting codes like $$Q, $$V, etc.
    return re.sub(r'\$\$[A-Z][^$]*', ' ', value).strip()

def parse_contributors(contrib_str):
    if not contrib_str:
        return []
    # Split the contributors by the pattern "$$Q"
    contributors = []
    parts = contrib_str.split('$$Q')
    for part in parts:
        if part.strip():
            contributors.append(clean_value(part))
    return contributors

def extract_value(item, field):
    """Extract a value from an item's field"""
    value_list = item.get(field, [])
    if value_list:
        return value_list[0].get('@value', '') if '@value' in value_list[0] else value_list[0].get('@id', '')
    return ""

def convert_nli_docs_to_dicts(docs):
    """Convert NLI API docs to a list of dictionaries"""
    results = []

    for item in docs:
        # Create a dictionary to store the extracted data
        result = {}

        # Extract basic metadata
        result['id'] = item.get('@id', '')

        # Title
        result['title'] = extract_value(item, 'http://purl.org/dc/elements/1.1/title')

        # Type
        result['type'] = extract_value(item, 'http://purl.org/dc/elements/1.1/type')

        # Record ID
        result['record_id'] = extract_value(item, 'http://purl.org/dc/elements/1.1/recordid')

        # Language
        result['language'] = extract_value(item, 'http://purl.org/dc/elements/1.1/language')

        # Subject
        result['subject'] = extract_value(item, 'http://purl.org/dc/elements/1.1/subject')

        # Publisher
        result['publisher'] = extract_value(item, 'http://purl.org/dc/elements/1.1/publisher')

        # Date
        result['date'] = extract_value(item, 'http://purl.org/dc/elements/1.1/non_standard_date')

        # Contributors - parse the complex string into a list and join with semicolons for CSV
        contrib_list = item.get('http://purl.org/dc/elements/1.1/contributor', [])
        if contrib_list:
            contrib_str = contrib_list[0].get('@value', '')
            result['contributors'] = '; '.join(parse_contributors(contrib_str))
        else:
            result['contributors'] = ''

        # Format
        result['format'] = extract_value(item, 'http://purl.org/dc/elements/1.1/format')

        # Thumbnail
        result['thumbnail'] = extract_value(item, 'http://purl.org/dc/elements/1.1/thumbnail')

        # Access Rights
        result['access_rights'] = extract_value(item, 'http://purl.org/dc/elements/1.1/accessRights')

        # Link to MARC
        result['link_to_marc'] = extract_value(item, 'http://purl.org/dc/elements/1.1/linkToMarc')

        results.append(result)

    return results

def save_to_csv(data, output_file):
    """Save data to CSV file"""
    if not data:
        print("No data to save")
        return False

    # Get all unique keys from all dictionaries
    fieldnames = set()
    for item in data:
        fieldnames.update(item.keys())

    fieldnames = sorted(list(fieldnames))

    # Make sure the directory exists
    dir_name = os.path.dirname(output_file)
    if dir_name and not os.path.exists(dir_name):
        os.makedirs(dir_name)

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

    print(f"Data saved to {output_file}")
    return True

def try_alternative_search(search_string, bulk_size=30):
    """Try different search formats if the main search fails"""
    alternative_formats = [
        # Try with explicit field searches
        f"dc.title,contains,{search_string}",
        f"dc.creator,contains,{search_string}",
        # Try changing the search operator
        f"any,exact,{search_string}",
        # Try with quotes
        f'any,contains,"{search_string}"'
    ]

    for query_format in alternative_formats:
        print(f"\nTrying alternative search format: {query_format}")
        try:
            # Construct the URL with the alternative format
            current_api_key = get_current_api_key()
            encoded_query = urllib.parse.quote(query_format)
            url = f"{BASE_URL}?api_key={current_api_key}&query={encoded_query}&bulkSize={bulk_size}&index=0&sort=rank"

            print(f"Request URL: {url}")
            response = requests.get(url)

            if response.status_code == 200:
                try:
                    result = response.json()
                    if isinstance(result, dict) and ('docs' in result or 'records' in result or 'results' in result):
                        # Adjust the structure if needed
                        if 'records' in result:
                            result['docs'] = result['records']
                        elif 'results' in result:
                            result['docs'] = result['results']

                        doc_count = len(result.get('docs', []))
                        print(f"Found {doc_count} documents with alternative format")
                        if doc_count > 0:
                            return result
                except Exception as e:
                    print(f"Error processing alternative response: {e}")

            print(f"Alternative format did not yield results (status code: {response.status_code})")

        except Exception as e:
            print(f"Error with alternative search: {e}")

    return None

def nli_search_to_csv(search_string, output_file=None, bulk_size=30):
    """
    Simple function to search NLI API and save results to CSV.

    Parameters:
    -----------
    search_string : str
        The search query to send to the NLI API
    output_file : str, optional
        Path to save the CSV file. If None, a filename will be generated based on the search string
    bulk_size : int, optional
        Number of results to retrieve (default: 30)

    Returns:
    --------
    str
        The path to the saved CSV file, or None if the search failed
    """
    # Generate output filename if not provided
    if not output_file:
        # Create a safe filename from the search string
        safe_filename = re.sub(r'[^\w\s-]', '', search_string).strip().replace(' ', '_')
        output_file = f"nli_search_{safe_filename}_{time.strftime('%Y%m%d_%H%M%S')}.csv"

    print(f"Starting with API key: {get_current_api_key()[:5]}...")

    # Search the NLI API
    result = search_nli(search_string, bulk_size)

    # If standard search fails, try alternative formats
    if not result:
        print("\nNo result returned. Trying alternative search formats...")
        result = try_alternative_search(search_string, bulk_size)
    # Check if result is a list (direct docs array) instead of a dictionary with 'docs' key
    elif isinstance(result, list):
        print(f"\nAPI returned a list of {len(result)} documents directly instead of a dictionary")
        # Convert to expected format
        result = {'docs': result}
    # Check if docs exists in the dictionary result
    elif isinstance(result, dict) and not result.get('docs', []):
        print("\nAPI returned a dictionary without 'docs' key. Trying alternative search formats...")
        result = try_alternative_search(search_string, bulk_size)

    if not result:
        print("No results found after trying all search methods")
        return None

    # Check if we have results
    if isinstance(result, dict) and 'docs' in result and result['docs']:
        print(f"Found {len(result['docs'])} results")
        # Convert to list of dictionaries
        data = convert_nli_docs_to_dicts(result['docs'])
        # Save to CSV
        if save_to_csv(data, output_file):
            return output_file
    elif isinstance(result, list) and result:
        print(f"Found {len(result)} results in list format")
        # Convert to list of dictionaries
        data = convert_nli_docs_to_dicts(result)
        # Save to CSV
        if save_to_csv(data, output_file):
            return output_file
    else:
        print("No documents found in the response after all attempts")

    return None

# Example usage
if __name__ == "__main__":
    # STEP 1: CHANGE THIS LINE TO YOUR SEARCH QUERY
    search_query = "990012026630205171"  # <-- ENTER YOUR SEARCH QUERY HERE

    # Optional parameters
    results_count = 50  # Number of results to retrieve
    csv_filename = None  # Output filename (None for auto-generated)

    # Run the search
    output_file = nli_search_to_csv(search_query, csv_filename, results_count)

    if output_file and os.path.exists(output_file):
        print(f"Successfully saved search results to: {output_file}")
    else:
        print("Search completed but no results were found or saved.")

Starting with API key: mZcuS...
Searching for: 990012026630205171
Request URL: https://api.nli.org.il/openlibrary/search?api_key=mZcuSKOCiaboR9ctxclfeMACDtXAgoYQL3cdTRWS&query=any,contains,990012026630205171&bulkSize=50&index=0&sort=rank
Response status code: 200
Response content type: application/json;charset=UTF-8
Response contains keys: Not a dictionary
Response is not a dictionary. Raw data (first 500 chars):
[{"@id": "https://www.nli.org.il/en/books/NNL_ALEPH990012026630205171", "http://purl.org/dc/elements/1.1/source": [{"@value": "The National Library of Israel"}], "http://purl.org/dc/elements/1.1/non_standard_date": [{"@value": "[\u05ea\u05e8\u05e2\"\u05d1]"}], "http://purl.org/dc/elements/1.1/identifier": [{"@value": "NNL_ALEPH71539165820005171"}], "http://purl.org/dc/elements/1.1/language": [{"@value": "heb"}], "http://purl.org/dc/elements/1.1/format": [{"@value": "(13) \u05e2\u05de\u05d5\u05d3...

API returned a list of 2 documents directly instead of a dictionary
Found 2 re